In [1]:
pacman::p_load(readr, rlist, stringr, purrr, tidyr, data.table, dplyr)

In [52]:
input_str <- "..........
...#......
..........
....a.....
..........
....a.....
..........
......#...
..........
.........."

In [53]:
input_str <- read_file("puzzle_input.txt")

In [54]:
input_str_clean <- str_split(input_str, "\n")[[1]] %>% list.filter(. != '')

ncols <- nchar(input_str_clean[1])
input_mat <- matrix(unlist(strsplit(input_str_clean, "")), nrow = length(input_str_clean), byrow = TRUE)

antenae_types <- input_str_clean %>% str_split("\n") %>% unlist() %>% str_extract_all("[^\\.|#]") %>% unlist() %>% unique()

# Part 1

In [ ]:
# nolint start
find_antinodes_for_antenae_type <- function(input_mat, current_type){
    antenae_positions <- which(input_mat == current_type, arr.ind = TRUE) %>% as.data.table %>% set_names(c("row", "col"))
    compare_coords <- tidyr::crossing(antenae_positions, antenae_positions %>% set_names(c("row2", "col2"))) %>% as.data.table %>%
                         .[!(row == row2 & col == col2)]

    compare_coords[, slope := (row2 - row) / (col2 - col)]
    compare_coords[, delta_x := col2 - col]
    compare_coords[, delta_y := row2 - row]

    compare_coords[, next_position_row := case_when(is.finite(slope) ~ row2 + delta_x * slope,
                                                    TRUE ~ row2 + delta_y)]
    compare_coords[, next_position_col := col2 + delta_x]

    # Need to round because of floating point errors with the slope calculation
    compare_coords[, IsValid := round(next_position_row, 2) %in% 1:nrow(input_mat) & round(next_position_col, 2) %in% 1:ncol(input_mat)]

    return(compare_coords)
}


In [176]:
all_results <- antenae_types %>% map_dfr(function(current_type){
    antinodes <- find_antinodes_for_antenae_type(input_mat, current_type)
    antinodes[, antenae_type := current_type]
})

In [177]:
all_results[(IsValid), .(next_position_row, next_position_col, IsValid)] %>%
    unique()%>% .[, sum(IsValid)]

[1] 371

# Part 2

In [199]:
# nolint start
find_antinodes_for_antenae_type <- function(input_mat, current_type){
    antenae_positions <- which(input_mat == current_type, arr.ind = TRUE) %>% as.data.table %>% set_names(c("row", "col"))
    compare_coords <- tidyr::crossing(antenae_positions, antenae_positions %>% set_names(c("row2", "col2"))) %>% as.data.table %>%
                         .[!(row == row2 & col == col2)]

    compare_coords[, delta_x := col - col2]
    compare_coords[, delta_y := row - row2]

    compare_coords <- tidyr::crossing(compare_coords, data.table(resonance = -ncol(input_mat):ncol(input_mat))) %>% as.data.table

    compare_coords[, next_position_row := row + delta_y * resonance]
    compare_coords[, next_position_col := col + delta_x * resonance]

    compare_coords[, IsValid := next_position_row %in% 1:nrow(input_mat) & next_position_col %in% 1:ncol(input_mat)]

    return(compare_coords)
}


In [200]:
all_results <- antenae_types %>% map_dfr(function(current_type){
    antinodes <- find_antinodes_for_antenae_type(input_mat, current_type)
    antinodes[, antenae_type := current_type]
})

In [201]:
all_results[(IsValid), .(next_position_row, next_position_col, IsValid)] %>%
    unique() %>% .[, sum(IsValid)]

[1] 1229